In [1]:
import os
home_dir = '/home/jupyter/'
ckpt_file = 'NMF.ckpt'
config_file = home_dir+'yaml/nmf_heart_config_default.yaml'
data_file = home_dir+'data/human_heart_atlas_hvg_scvi_rec.h5ad'

In [2]:
path = os.getcwd()
if ckpt_file in os.listdir(path):
    print("trained checkpoint saved")
else:
    # fit cnmf
    !cellarium-ml nmf fit -c $config_file

Seed set to 0
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name     | Type              | Params
-----------------------------------------------
0 | pipeline | CellariumPipeline | 1     
-----------------------------------------------
1         Trainable params
0         Non-trainable params
1         Total params
0.000     Total estimated model params size (MB)
Epoch 0: 100%|█████████████████████████| 67/67 [01:27<00:00,  0.77it/s, v_num=7]


In [3]:
output_dir = home_dir+'cnmf_output/'
output_prefix = 'NMF'

if os.path.exists(output_dir):
    print("output directary already existed")
else:
    !mkdir $output_dir

output directary already existed


In [4]:
import sys
parent_dir = os.path.dirname('/home/jupyter/')
sys.path.append(parent_dir)

import warnings
warnings.filterwarnings("ignore")

In [5]:
import yaml
import torch
import anndata
import tempfile
import numpy as np
import pandas as pd
import scanpy as sc

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [6]:
import cudf
import cuml

In [7]:
from cellarium.ml.core import CellariumPipeline, CellariumModule
from cellarium.ml.utilities.data import AnnDataField, densify
from cellarium.ml.core.datamodule import CellariumAnnDataDataModule
from cellarium.ml.data import (
    DistributedAnnDataCollection,
    IterableDistributedAnnDataCollectionDataset,
)

from cellarium.ml.models.nmf import calculate_rec_error, get_embeddding, update_consensusD

In [8]:
class AutosizedDistributedAnnDataCollection(DistributedAnnDataCollection):

    def __init__(self, *args, **kwargs):
        # I'm being lazy here and doing something real ugly
        # I want it to take the shard_size from the first file
        try:
            # this allows super to find the list of filenames
            super().__init__(*args, **kwargs)
        except AssertionError:
            try:
                # this allows super to create the cache
                kwargs.pop("shard_size")
                kwargs = kwargs | {"shard_size": 10000}
                super().__init__(*args, **kwargs)
            except AssertionError:
                pass
            # load first file and cache it
            adata0 = self.cache[self.filenames[0]] = read_h5ad_file(self.filenames[0])
            # pull shard_size from that file
            kwargs.pop("shard_size")
            kwargs = kwargs | {"shard_size": len(adata0)}
            # finally initialize for real
            super().__init__(*args, **kwargs)

def get_pretrained_model_as_pipeline(
    trained_model: str = "../", 
    transforms: list[torch.nn.Module] = [],
    device: str = "cuda",
) -> CellariumPipeline:

    model = CellariumModule.load_from_checkpoint(trained_model).model

    # insert the trained model params
    model.to(device)
    model.eval()

    # construct the pipeline
    pipeline = CellariumPipeline(transforms + [model])

    return pipeline

def get_dataset_from_anndata(
    adata: anndata.AnnData | str, 
    batch_size: int = 2048, 
    shard_size: int | None = None, 
    shuffle: bool = False, 
    shuffle_seed: int = 0, 
    drop_last_indices: bool = False,
):
    """
    Get IterableDistributedAnnDataCollectionDataset from an AnnData object or h5ad file specifier.

    Args:
        adata: AnnData object or h5ad file, allowing brace notation for several files.
        batch_size: Batch size.
        shard_size: Shard size.
        shuffle: Whether to shuffle the dataset.
        seed: Random seed.
        drop_last: Whether to drop the last incomplete batch.

    Returns:
        IterableDistributedAnnDataCollectionDataset.
    """

    if isinstance(adata, anndata.AnnData):
        tmpfile = tempfile.mkstemp(suffix='.h5ad')
        adata.write(tmpfile[1])
        file = tmpfile[1]
    else:
        file = adata

    dadc = AutosizedDistributedAnnDataCollection(
        file,
        shard_size=shard_size,
        max_cache_size=1,
    )

    dataset = IterableDistributedAnnDataCollectionDataset(
        dadc,
        batch_keys={
            "x_ng": AnnDataField(attr="X", convert_fn=densify),
            "var_names_g": AnnDataField(attr="var_names"),
        },
        batch_size=batch_size,
        shuffle=shuffle,
        shuffle_seed=shuffle_seed,
        drop_last_indices=drop_last_indices,
    )
    return dataset

In [9]:
# load trained pipeline
pipeline = get_pretrained_model_as_pipeline(
    trained_model=os.getcwd()+'/'+ckpt_file,
)

In [10]:
# get a dataset object
dataset = get_dataset_from_anndata(
    data_file, 
    batch_size=2048,
    shard_size=665234,
    shuffle=False,
    shuffle_seed=0,
    drop_last_indices=False,
)

In [ ]:
# calculate consensus D for all Ks
density_threshold = 0.2
local_neighborhood_size = 0.3
consensus_stat = update_consensusD(pipeline,
                                   density_threshold = density_threshold, 
                                   local_neighborhood_size = local_neighborhood_size)

In [14]:
# save gene loading (hvg) for all K
k_range = pipeline[-1].k_range
for k in k_range:
    D_kg = getattr(pipeline[-1], f"D_{k}_kg")
    D_kg = D_kg.cpu().numpy()
    gene_loading_file=output_dir+output_prefix+ ".k=%d.gene_loadings.txt" % (k)
    np.savetxt(gene_loading_file, D_kg, delimiter='\t')

In [ ]:
for k in k_range:
    print('k='+str(k))
    
    sc.set_figure_params(scanpy=True, dpi=75, dpi_save=75, vector_friendly=True)
    sns.set_style("ticks")
    sns.clustermap(consensus_stat[k]['topk_euc_dist'].cpu().numpy())
    plt.show()
    
    sns.histplot(consensus_stat[k]['local_neigh_dist'].cpu().numpy())
    ymax = plt.gca().get_ylim()[1]
    plt.vlines(density_threshold, ymin=0, ymax=ymax, color='Red')
    plt.xlabel('Mean distance to k nearest neigbors')
    plt.ylim(0, ymax)
    
    plt.show()

In [ ]:
# calculate reconstruction error
rec_errors = calculate_rec_error(dataset, pipeline)

In [20]:
silhouette_scores = {}
for k in k_range:
    silhouette_scores[k] = consensus_stat[k]['stability']
eval_metrics = pd.DataFrame.from_dict(silhouette_scores, orient='index')
eval_metrics.columns = ['stability']
eval_metrics['rec_error'] = rec_errors

In [ ]:
# plot stability and reconstruction error
sc.set_figure_params(scanpy=True, dpi=75, dpi_save=75, vector_friendly=True, fontsize=10)
sns.set_style("ticks")
plt.plot(eval_metrics.index, eval_metrics['stability'], 
         color='Red', label='Stabibility')
plt.legend(loc="upper right")
plt.twinx()
plt.plot(eval_metrics.index, eval_metrics['rec_error'], 
         color='Blue', label='Reconstruction error')
plt.legend(loc="upper left")

In [18]:
# get cell loading with best K
the_best_k = 20
obsm_key_added = 'X_nmf'
adata = get_embeddding(dataset, pipeline, 
                       the_best_k=the_best_k, obsm_key_added=obsm_key_added)

# save cell loading with the best k
cell_loading_file=output_dir+output_prefix+ ".k=%d.cell_loadings.txt" % (the_best_k)
np.savetxt(cell_loading_file, adata.obsm[obsm_key_added], delimiter='\t')

# save gene loading with the best k (full transcriptome)
gene_loading_file=output_dir+output_prefix+ ".k=%d.full_gene_loadings.txt" % (the_best_k)
full_D_kg = getattr(pipeline[-1], f"full_D_{the_best_k}_kg")
full_D_kg = full_D_kg.cpu().numpy()
np.savetxt(gene_loading_file, full_D_kg, delimiter='\t')

In [23]:
# visualization with UMAP
reducer = cuml.UMAP(
    n_neighbors=15,
    n_components=2,
    n_epochs=500,
    min_dist=0.15,
    metric='cosine'
)
embedding = reducer.fit_transform(adata.obsm['X_nmf'])
adata.obsm['X_nmf_umap'] = embedding

In [ ]:
sc.set_figure_params(scanpy=True, dpi=75, dpi_save=75, vector_friendly=True, fontsize=7.5)
sc.pl.embedding(adata, basis='nmf_umap', color=['cell_type', 'cell_state', 'batch'], ncols=1)